In [2]:
import numpy as np
import json
import urllib
import geopandas as gpd

%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [3]:
#https://github.com/mfalcon/yomemudo/blob/master/api.py
#lat y -34
#long x -58 
def queryUSIG(x,y,destino,tipo_transporte):
    origen = inverseGeocode(x = x,y = y)
    if origen[0] == None:
        return np.nan
    else:
        resultado = get_trips(o_x = origen[0], o_y = origen[1], d_x = destino[0], d_y =destino[1],
                              tipo_transporte = tipo_transporte)
        return resultado['tiempo']
    
def inverseGeocode(x,y):
    url = 'http://ws.usig.buenosaires.gob.ar/geocoder/2.2/reversegeocoding?x={0}&y={1}'.format(x,y)
    response = urllib.urlopen(url)
    data = json.loads(response.read()[1:-1])
    
    return (data['puerta_x'],data['puerta_y'])

def get_trips(o_x, o_y, d_x, d_y, tipo_transporte):
    #version resumida
    #url = "http://recorridos.usig.buenosaires.gob.ar/2.0/consultar_recorridos?tipo={4}&origen={0}%2C{1}&destino={2}%2C{3}".format(o_x, o_y, d_x, d_y, tipo_transporte)  
    #completa con opciones explicitas
    url = "http://recorridos.usig.buenosaires.gob.ar/2.0/consultar_recorridos?tipo={4}&gml=true&precargar=3&opciones_caminata=800&opciones_medios_colectivo=true&opciones_medios_subte=true&opciones_medios_tren=true&opciones_prioridad=avenidas&opciones_incluir_autopistas=true&opciones_cortes=true&max_results=1&origen={0}%2C{1}&destino={2}%2C{3}".format(o_x, o_y, d_x, d_y, tipo_transporte);
    response = urllib.urlopen(url)
    viaje = json.loads(response.read())
    try:
        viaje = json.loads(viaje['planning'][0])
        return viaje
    except IndexError:
        return np.nan



In [4]:
#fijar destino arbitrario en Peron 450
destino = inverseGeocode(x = -58.373022,y=-34.605537)

In [5]:
# cargar radios de buenos aires
radios = gpd.read_file('data/radioCentroids.shp')
radios = radios.loc[:,['CO_FRAC_RA','tiemposMin','x','y']]
radios.head()

,CO_FRAC_RA,tiemposMin,x,y
0,1_1_1,21.533333,-58.368253,-34.587126
1,1_12_1,10.800000,-58.385199,-34.605043
2,1_12_10,3.083333,-58.376849,-34.605250
3,1_12_11,3.316667,-58.371612,-34.605709
4,1_12_2,11.433333,-58.383781,-34.604953


In [7]:
index = []
tiemposBici = []
for i in range(radios.shape[0]):
    print '\r',"%",str((i+1)*100./radios.shape[0])[:4],
    index.append(i)
    tiemposBici.append(queryUSIG(x=radios.x[i],y=radios.y[i],destino=destino,tipo_transporte='bici'))

    radios['tiemposBici'] = tiemposBici

% 100.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [17]:
tiemposAuto = [queryUSIG(x=radios.x[i],y=radios.y[i],destino=destino,tipo_transporte='auto') for i in range(radios.shape[0])]
radios['tiemposAuto'] = tiemposAuto


In [30]:
index = []
tiemposTransp = []
for i in range(radios.shape[0]):
    print '\r',"%",str((i+1)*100./radios.shape[0])[:4],
    index.append(i)
    tiemposTransp.append(queryUSIG(x=radios.x[i],y=radios.y[i],destino=destino,tipo_transporte='transporte'))


% 97.3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

IOError: ('http protocol error', 0, 'got a bad status line', None)

3459

In [37]:


for i in range(3459,radios.shape[0]):
    print '\r',"%",str((i+1)*100./radios.shape[0])[:4],
    index.append(i)
    tiemposTransp.append(queryUSIG(x=radios.x[i],y=radios.y[i],destino=destino,tipo_transporte='transporte'))


% 100.                                                                                              


In [39]:
len(tiemposTransp)

3554

In [40]:
radios.shape

(3554, 6)

In [41]:
radios['tiemposTransp'] = tiemposTransp

In [42]:
radios.head()

,CO_FRAC_RA,tiemposMin,x,y,tiemposBici,tiemposAuto,tiemposTransp
0,1_1_1,21.533333,-58.368253,-34.587126,15.0,10.0,14.0
1,1_12_1,10.800000,-58.385199,-34.605043,12.0,10.0,8.0
2,1_12_10,3.083333,-58.376849,-34.605250,4.0,6.0,8.0
3,1_12_11,3.316667,-58.371612,-34.605709,1.0,2.0,2.0
4,1_12_2,11.433333,-58.383781,-34.604953,9.0,7.0,8.0


In [66]:
valores = radios.loc[:,['tiemposBici','tiemposAuto','tiemposTransp']]
minvalues = valores.values.min(axis=1)
better=[ ]

for i in range(radios.shape[0]):
    if np.isnan(maxvalues[i]):
        mejor = np.nan
    else:
        mejor = radios.columns[radios.loc[i] == minvalues[i]].values[0]
    better.append(mejor)
    
#radios['masRapido'] = better

In [68]:
radios['mejorMod'] = better

/* CSS - Cascading Style Sheet */
/* Palette color codes */
/* Palette URL: http://paletton.com/#uid=30k0l0kllllwh+PqIvZfYaHaA04 */

.color-primary-0 { color: #AA6039 }	/* Main Primary color */
.color-primary-1 { color: #FF5900 }
.color-primary-2 { color: #FF742A }
.color-primary-3 { color: #553A2B }
.color-primary-4 { color: #000000 }

.color-secondary-1-0 { color: #28546C }	/* Main Secondary color (1) */
.color-secondary-1-1 { color: #0DAAFE }
.color-secondary-1-2 { color: #2275A2 }
.color-secondary-1-3 { color: #1D2D36 }
.color-secondary-1-4 { color: #000000 }

.color-secondary-2-0 { color: #378B2E }	/* Main Secondary color (2) */
.color-secondary-2-1 { color: #18FF00 }
.color-secondary-2-2 { color: #33D122 }
.color-secondary-2-3 { color: #264623 }
.color-secondary-2-4 { color: #000000 }




In [69]:
radiosFinal = gpd.read_file('data/radios.shp')
radiosFinal = radiosFinal.drop(['x','y','tiemposMin'],axis=1).merge(radios, on = ['CO_FRAC_RA'])
radiosFinal.head()


,CO_FRAC_RA,geometry,tiemposMin,x,y,tiemposBici,tiemposAuto,tiemposTransp,mejorMod
0,1_1_1,"POLYGON ((108330.8437499999 104181.0937500013,...",21.533333,-58.368253,-34.587126,15.0,10.0,14.0,tiemposAuto
1,1_12_1,"POLYGON ((107042.4609375 102801.421875, 107058...",10.800000,-58.385199,-34.605043,12.0,10.0,8.0,tiemposTransp
2,1_12_10,"POLYGON ((107697.2187500009 102707.15625, 1078...",3.083333,-58.376849,-34.605250,4.0,6.0,8.0,tiemposBici
3,1_12_11,"POLYGON ((108748.546875 102668.9453125009, 108...",3.316667,-58.371612,-34.605709,1.0,2.0,2.0,tiemposBici
4,1_12_2,"POLYGON ((107169.9375 102810.3671875, 107300.3...",11.433333,-58.383781,-34.604953,9.0,7.0,8.0,tiemposAuto


In [70]:

radiosFinal.to_file('radiosFinal')